In [571]:
import speech_recognition as sr
import pyttsx3
import datetime
import subprocess
import os
import shutil
import re
import webbrowser
import pygetwindow as gw

In [572]:
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)

In [573]:
current_directory = os.getcwd()

In [574]:
symbols_to_replace = {
    "dot": ".",
    "hyphen": "-",
    "underscore": "_",
    "dollar": "$",
    "Hashtag": "#",
    "at_the_rate": "@",
    "ampersand": "&",
    "percent": "%",
    "plus": "+",
    "equals": "=",
    "exclamation": "!",
    "question": "?",
    "asterisk": "*",
    "left_parenthesis": "(",
    "right_parenthesis": ")",
    "left_square_bracket": "[",
    "right_square_bracket": "]",
    "left_curly_brace": "{",
    "right_curly_brace": "}",
    "pipe": "|",
    "backslash": "\\",
    "forward_slash": "/",
    "tilde": "~",
    "grave_accent": "`",
    "double_quote": '"',
    "single_quote": "'",
    "colon": ":",
    "space": " ",
    "zero": "0",
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9"
}

In [575]:
clipboard = {"path" : None, "cut" : False}

In [576]:
def replace_words_with_symbols(text):

    words = text.split()
    replaced_words = [symbols_to_replace.get(word.lower(), word) for word in words]
    return ''.join(replaced_words)

In [577]:
def find_files_with_same_base_name(base_name):
    matched_files = []
    for item in os.listdir(current_directory):
        if os.path.isfile(os.path.join(current_directory, item)):
            base, ext = os.path.splitext(item)
            if base.lower() == base_name.lower():
                matched_files.append(item)
    return matched_files

In [578]:
def prompt_for_extension(base_name, files):
    speak(f"Multiple files found with the name {base_name} but different extensions.")
    print(f'Multiple files found : ')
    for i, file in enumerate(files):
        print(f'{i + 1}. {file}')
    speak("Please specify the number of the file you want to open.")
    user_choice = input("Enter the number : ")
    try:
        index = int(user_choice) - 1
        if 0 <= index < len(files):
            return files[index]
        else:
            speak("Invalid choice. Please try again.")
            return None
    except ValueError:
        speak("Invalid input. Please enter a number.")
        return None

In [579]:
def open_file(file_path):
    global current_directory, found
    try:
        if os.path.exists(file_path) and os.path.isfile(file_path):
            os.startfile(file_path)
            current_directory = os.path.dirname(file_path)
            speak(f'Opening file {item_name}')
            found = True
    
    except Exception as e:
        speak(f'Error opening {item_name}: {e}')
        print(f'Error opening {item_name}: {e}', '\n')
        

In [580]:
def open_folder(folder_path):
    global current_directory, found
    try:
        if os.path.exists(folder_path) and os.path.isdir(folder_path):
            os.chdir(folder_path)
            os.startfile('.')
            current_directory = folder_path
            speak(f'Opening folder {matching_folders[0]}')
            found = True

    except Exception as e:
        speak(f'Error opening {matching_folders[0]}: {e}')
        print(f'Error opening {matching_folders[0]}: {e}', '\n')

In [581]:
def close_window(window_title):
    try:
        window = gw.getWindowsWithTitle(window_title)[0]
        window.close()
        speak(f"Closed {window_title} successfully.")
    except IndexError:
        speak(f"No window with title {window_title} found.")

In [582]:
def list_files():
    global current_directory
    try:
        contents = os.listdir(current_directory)
        files = [item for item in contents if os.path.isfile(os.path.join(current_directory, item))]
        if files:
            speak("The files in the current directory are:")
            for file in files:
                print(file, '\n')
        else:
            speak("There are no files in the current directory.")
    except Exception as e:
        speak(f"Error listing files: {str(e)}")
        print(f"Error listing files: {str(e)}", '\n')

In [583]:
def list_folders():
    global current_directory
    try:
        contents = os.listdir(current_directory)
        folders = [item for item in contents if os.path.isdir(os.path.join(current_directory, item))]
        if folders:
            speak("The folders in the current directory are:")
            for folder in folders:
                print(folder, '\n')
        else:
            speak("There are no folders in the current directory.")
    except Exception as e:
        speak(f"Error listing folders: {str(e)}")
        print(f"Error listing folders: {str(e)}, '\n'")

In [584]:
def create_folder(folder_name):
    global found
    try:
        os.mkdir(folder_name)
        speak(f"Folder '{os.path.basename(folder_name)}' created successfully.")
        found = True
    except OSError as e:
        speak(f"Failed to create folder '{os.path.basename(folder_name)}': {e}")

In [585]:
def delete_file(file_path):
    global found
    try:
        os.remove(file_path)
        speak(f"File '{item_name}' deleted successfully.")
        found = True
    except Exception as e:
        speak(f"Failed to delete {item_name}: {str(e)}")

In [586]:
def delete_folder(folder_path):
    global found
    try:
        shutil.rmtree(folder_path)
        speak(f"File '{matching_folders}' deleted successfully.")
        found = True
    except Exception as e:
        speak(f"Failed to delete {matching_folders}: {str(e)}")

In [587]:
def rename_item(old_name, new_name, item_type):
    global current_directory, found
    old_path = os.path.join(current_directory, old_name)
    new_path = os.path.join(current_directory, new_name)
    
    if os.path.exists(new_path):
        speak(f"A {item_type} with the name '{new_name}' already exists in the current directory.")
        return

    try:
        os.rename(old_path, new_path)
        speak(f"Successfully renamed {item_type} '{old_name}' to '{new_name}'.")
        found = True
    except Exception as e:
        speak(f"Failed to rename {item_type} '{old_name}' to '{new_name}': {str(e)}")

In [588]:
def copy_item(item_name):
    global current_directory, found, clipboard
    source_path = os.path.join(current_directory, item_name)

    try:
        if os.path.exists(source_path):
            clipboard['path'] = source_path
            clipboard['cut'] = False
            speak(f"{item_name} copied successfully.")
            found = True

        else:
            speak(f"Item '{item_name}' does not exist in the current directory.")

    except Exception as e:
        speak(f"Failed to copy '{item_name}': {str(e)}")
        clipboard['path'] = None
        clipboard['cut'] = False

In [589]:
def cut_item(item_name):
    global current_directory, found, clipboard
    source_path = os.path.join(current_directory, item_name)

    try:
        if os.path.exists(source_path):
            clipboard['path'] = source_path
            clipboard['cut'] = True
            speak(f"{item_name} cut successfully.")
            found = True

        else:
            speak(f"Item '{item_name}' does not exist in the current directory.")

    except Exception as e:
        speak(f"Failed to cut '{item_name}': {str(e)}")
        clipboard['path'] = None
        clipboard['cut'] = False

In [590]:
def paste_item(destination_path):
    global clipboard, found
    source_path = clipboard["path"]
    cut_mode = clipboard["cut"]

    if source_path:
        item_name = os.path.basename(source_path)
        destination = os.path.join(destination_path, item_name)
        try:
            if os.path.exists(source_path):
                if cut_mode:
                    shutil.move(source_path, destination)
                    clipboard["path"] = None
                    clipboard["cut"] = False
                    speak(f"{item_name} moved successfully.")
                else:
                    if os.path.isdir(source_path):
                        shutil.copytree(source_path, destination)
                    else:
                        shutil.copy2(source_path, destination)
                    clipboard["path"] = None
                    speak(f"{item_name} copied successfully.")
                found = True
            else:
                speak("No item is currently in the clipboard.")
        except Exception as e:
            speak(f"Failed to paste : {str(e)}")
            clipboard['path'] = None
            clipboard['cut'] = False

In [591]:
def speak(audio):
    engine.say(audio)
    engine.runAndWait()

In [592]:
greeting_dict = {'hello' : 'Hello', 'hi' : 'Hi', 'hey' : 'Hey'}

In [593]:
def is_valid_note(greet_dict, voice_text):
    for word in voice_text.split():
        if word.lower() in greet_dict:
            return True
    return False

In [594]:
def greetme():
    hour = int(datetime.datetime.now().hour)
    if 0 <= hour < 12:
        speak('Good Morning Sir')

    elif 12 <= hour < 18:
        speak('Good Afternoon Sir')

    else:
        speak('Good Evening Sir')

    speak('How may I assist you?')

In [595]:
def takecommand():
        
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print('Listening...')
        r.pause_threshold = 0.8
        r.adjust_for_ambient_noise(source, duration = 1)  # Adjust for background noise

        try:
            audio = r.listen(source, timeout = 10)  # Set a timeout for listening
            print('Recognizing...')

            voice_text = r.recognize_google(audio, language = 'en-in')
            print(f'User said : {voice_text}\n')
            return voice_text.lower()

        except sr.WaitTimeoutError:
            print('Listening timed out, Please try again.', '\n')
            return 'None'

        except sr.UnknownValueError:
            print('Sorry, I did not understand what you said.', '\n')
            return 'None'
    
        except sr.RequestError:
            print('Sorry, I could not request results. Please check your internet connection.', '\n')
            return 'None'
    
        except Exception as e:
            print(f'Error: {str(e)}', '\n')
            return 'None'

In [596]:
def wake_word_detection():
    while True:
        speak("Please speak the wake word to activate Nexus")
        print("Listening for the wake word...\n")
        query = takecommand().lower()
        if not 'wake up nexus' in query:
            print("Failed to detect the wake word\n")
        else:
            greetme()
            break

In [597]:
if __name__ == '__main__':
    first_command = True
    wake_word_detection()

    while True:
        voice_text = takecommand()

        if is_valid_note(greeting_dict, voice_text):
            speak('Hi. How can I help you?')
            continue
        
        first_command = False

        command_parts = voice_text.lower().split()

        if 'file' in command_parts or 'folder' in command_parts or 'paste' in command_parts:
            if 'paste' in command_parts:
                action = 'paste'
            else:
                item_type = 'file' if 'file' in command_parts else 'folder'
                action = None
                found = False

            for action_keyword in ['open', 'close', 'create', 'delete', 'rename', 'copy', 'cut']:
                if action_keyword in command_parts:
                    action = action_keyword
                    break

            if action and action != 'paste':
                try:
                    item_index = command_parts.index(item_type)
                    item_name = ' '.join(command_parts[item_index + 1:]).strip()
                    item_name = replace_words_with_symbols(item_name).lower()
                except ValueError:
                    speak(f"Please specify the {item_type} name.")
                    continue

                matching_items = [item for item in os.listdir(current_directory) if item.lower() == item_name]
                matching_files = find_files_with_same_base_name(item_name)
                matching_folders = [item for item in matching_items if os.path.isdir(os.path.join(current_directory, item))]

            if action == 'open':

                if item_type == 'file' and matching_files:

                    if len(matching_files) == 1:
                        open_file(os.path.join(current_directory, matching_files[0]))
                    else:
                        selected_file = prompt_for_extension(item_name, matching_files)
                        if selected_file:
                            open_file(os.path.join(current_directory, selected_file))

                elif item_type == 'folder' and matching_folders:
                    open_folder(os.path.join(current_directory, matching_folders[0]))

                else:
                    if item_type == 'file':
                        speak(f'Sorry, I could not find the {item_type} {item_name}. Please provide a more specific name.')

                    elif item_type == 'folder':
                        speak(f'Sorry, I could not find the {item_type} {item_name}. Please provide a more specific name.')

            elif action == 'close':
                close_window(item_name)

            elif action == 'create' and item_type == 'folder':

                new_folder_path = os.path.join(current_directory, item_name)

                if not os.path.exists(new_folder_path):
                    create_folder(new_folder_path)
            
                else:
                    speak(f"Folder '{item_name}' already exists.")

            elif action == 'delete':

                if os.path.join(current_directory, item_name):
                    confirmation = input(f"Are you sure you want to delete '{item_name}'? (yes/no): ").lower()

                    if confirmation == 'yes':

                        if item_type == 'file' and matching_files:
                            if len(matching_files) == 1:
                                delete_file(os.path.join(current_directory, matching_files[0]))
                            else:
                                selected_file = prompt_for_extension(item_name, matching_files)
                                if selected_file:
                                    delete_file(os.path.join(current_directory, selected_file))

                        elif item_type == 'folder' and matching_folders:
                            delete_folder(os.path.join(current_directory, matching_folders[0]))

                    else:
                        speak("Deletion canceled.")

                else:
                    speak(f"Could not find the {item_type} '{item_name}' to delete.")

            elif action == 'rename':
                try:
                    as_index = command_parts.index('as')
                    old_name = ' '.join(command_parts[item_index + 1 : as_index]).strip()
                    new_name = ' '.join(command_parts[as_index + 1 : ]).strip()
                    old_name = replace_words_with_symbols(old_name).lower()
                    new_name = replace_words_with_symbols(new_name).lower()
                    rename_file = find_files_with_same_base_name(old_name)
                    rename_folder = os.path.isdir(os.path.join(current_directory, old_name))

                    if item_type == 'file' and rename_file:

                        if len(rename_file) == 1:
                            rename_item(old_name, new_name, 'file')
                        else:
                            selected_file = prompt_for_extension(old_name, rename_file)
                            if selected_file:
                                rename_item(old_name, new_name, 'file')

                    elif item_type == 'folder' and rename_folder:
                        rename_item(old_name, new_name, 'folder')

                    else:
                        speak(f"{item_type.capitalize()} '{old_name}' does not exist in the current directory.")

                except ValueError:
                    speak("Please specify the new name after 'as'.")

            elif action == 'copy':

                if item_type == 'file' and matching_files:

                    if len(matching_files) == 1:
                        clipboard = {"path" : os.path.join(current_directory, matching_files[0]), "cut" : False}
                        speak(f"{item_name} copied to clipboard.")
                    else:
                        selected_file = prompt_for_extension(item_name, matching_files)
                        if selected_file:
                            clipboard = {"path" : os.path.join(current_directory, selected_file), "cut" : False}
                            speak(f"{selected_file} copied to clipboard.")

                elif item_type == 'folder' and matching_folders:
                    clipboard = {"path" : os.path.join(current_directory, matching_folders[0]), "cut" : False}
                    speak(f"{item_name} copied to clipboard.")

                else:
                    speak(f"Could not find the {item_type} '{item_name}' to copy.")

            elif action == 'cut':

                if item_type == 'file' and matching_files:

                    if len(matching_files) == 1:
                        clipboard = {"path" : os.path.join(current_directory, matching_files[0]), "cut" : True}
                        speak(f"{item_name} cut to clipboard.")
                    else:
                        selected_file = prompt_for_extension(item_name, matching_files)
                        if selected_file:
                            clipboard = {"path" : os.path.join(current_directory, selected_file), "cut" : True}
                            speak(f"{selected_file} cut to clipboard.")

                elif item_type == 'folder' and matching_folders:
                    clipboard = {"path" : os.path.join(current_directory, matching_folders[0]), "cut" : True}
                    speak(f"{item_name} cut to clipboard.")

                else:
                    speak(f"Could not find the {item_type} '{item_name}' to cut.")

            elif action == 'paste':

                if clipboard["path"]:
                    item_name = os.path.basename(clipboard["path"])
                    destination_path = current_directory

                    paste_item(destination_path)

                    clipboard = {"path" : None, "cut" : False}
                else:
                    speak("Clipboard is empty. Copy or cut an item first.")

            else:
                speak(f"Please specify the action you want to perform on the {item_type}.")
                found = True

        elif 'open' in voice_text.lower():
            found = False

            # Opening Desktop
            if 'desktop' in voice_text.lower():
                os.system('explorer shell:Desktop')
                current_directory = os.path.join(os.path.expanduser("~"), 'Desktop')
                speak('Opening Desktop')
                found = True

            # Opening This PC or My PC
            elif 'this pc' in voice_text.lower() or 'my pc' in voice_text.lower():
                if 'this pc' in voice_text.lower():
                    pc_name = 'This PC'
                else:
                    pc_name = 'my pc'

                current_directory = os.path.abspath(os.sep)
                os.system('explorer shell:MyComputerFolder')
                speak(f'Opening {pc_name}')
                found = True

            # Opening Drives
            elif 'drive' in voice_text.lower():
                # Check if the user mentioned a drive letter directly (e.g., 'c')
                match = re.search(r'\b([a-zA-Z])\b', voice_text)

                if match:
                    drive = match.group(1)

                    if re.search(r'\bdrive\s*[a-zA-Z]\b', voice_text.lower()):
                        drive_phrase = f'drive {drive}'
                    else:
                        drive_phrase = f'{drive} drive'

                # Construct the drive path
                    drive_path = f"{drive.upper()}:\\"
                    if os.path.exists(drive_path):
                        os.startfile(drive_path)
                        current_directory = drive_path
                        speak(f'Opening {drive_phrase}')
                        found = True
                    else:
                        speak(f"Sorry, I couldn't find {drive_phrase}")
                else:
                    speak("Sorry, I couldn't recognize the drive letter in your command. Can you please repeat?")
                    found = True
                    continue

                found = True

            # Opening Websites
            else:
                sites = {"youtube": "https://www.youtube.com", "google search": "https://www.google.com"}
                for site, url in sites.items():
                    if f"open {site}" in voice_text.lower():
                        speak(f"Launching {site} sir...")
                        webbrowser.open(url)
                        found = True
                        break

                # Opening Files/Folders from Current Directory
                if not found:
                    speak('Sorry, I could not understand your command.')

        elif 'time' in voice_text:
            current_time = datetime.datetime.now()
            hour = int(current_time.strftime('%I'))
            minute = int(current_time.strftime('%M'))
            second = int(current_time.strftime('%S'))
            meridiem = current_time.strftime('%p')

            time_message = ''
            if hour == 1:
                time_message += f'{hour} hour '
            else:
                time_message += f'{hour} hours '

            if minute == 1:
                time_message += f'{minute} minute '
            else:
                time_message += f'{minute} minutes '

            if second == 1:
                time_message += f'{second} second'
            else:
                time_message += f'{second} seconds'

            time_message += f'{meridiem}'

            speak(f'Sir the time is {time_message}')

        elif 'directory' in voice_text.lower():
            if 'current' in voice_text.lower():
            
                if 'list files' in voice_text.lower():
                    list_files()

                elif 'list folders' in voice_text.lower():
                    list_folders()

                else:
                    speak('The current directory is as follows: ')
                    print(current_directory, '\n')

            elif 'go back' in voice_text.lower():
                if current_directory != os.path.expanduser("~"):
                    previous_directory = os.path.dirname(current_directory)
                    os.chdir(previous_directory)
                    current_directory = previous_directory
                    speak('Navigating back to the previous directory.')
                else:
                    speak('You are already at the root directory.')

        elif 'navigate to' in voice_text.lower():
            folder_name = None
            parts = voice_text.lower().split()

            directory_index = parts.index('to')
            folder_name = ' '.join(parts[directory_index + 1:])

            folder_name = replace_words_with_symbols(folder_name)

            # Check if the folder exists in the directory path, not just in the current directory
            directory_parts = current_directory.split(os.path.sep)
            for i, part in enumerate(directory_parts):
                if folder_name.lower() in part.lower():
                    # Construct the new path by joining the parts up to the matching part
                    new_path = os.path.sep.join(directory_parts[:i + 1])
                    os.chdir(new_path)
                    current_directory = new_path
                    speak(f"Navigating to folder {part}.")
                    break
            else:
                speak(f"Folder '{folder_name}' not found in the directory path.")


        elif 'contents' in voice_text.lower():
            speak('Checking current directory contents...')
            contents = os.listdir(current_directory)

            if not contents:
                speak('There are no files or folders in the current directory.')

            else:
                speak('The current directory contains are as follows: ')
                for item in contents:
                    print(item, '\n')
    
        elif 'shutdown' in voice_text:
            speak('Okay Sir, please call me when you need me')
            break
        
        else:
            first_command = True
            speak('Sorry, I could not understand your command. Can you please repeat?')

        if not first_command:
            speak('How may I assist you further?')

Listening for the wake word...

Listening...
Recognizing...
User said : wake up Nexus

Listening...
Recognizing...
User said : what is the time

Listening...
Recognizing...
User said : current directory

d:\SAMRAT\Programming\Machine Learning Course\AI Voice Assistant\Demo 

Listening...
Recognizing...
User said : go back to the previous directory

Listening...
Recognizing...
User said : open the folder demo

Listening...
Recognizing...
User said : close the folder demo

Listening...
Recognizing...
User said : close the folder AI space voice space assistant

Listening...
Recognizing...
Sorry, I did not understand what you said. 

Listening...
Recognizing...
User said : open YouTube

Listening...
Recognizing...
User said : open Google search

Listening...
Recognizing...
Sorry, I did not understand what you said. 

Listening...
Recognizing...
User said : list files in the current directory

Listening...
Recognizing...
User said : list folders in the current directory

Listening...
Recogn